In [1]:
import os
from tqdm import tqdm

import config
import modules.classification_dataloaders as classification_data_loader
import modules.dataloaders as detection_data_loader
import modules.metrics as metrics

import numpy as np
import torch
import torchmetrics
from torchmetrics.detection.mean_ap import MeanAveragePrecision

In [2]:
import onnx
import onnxruntime

# Check Models

## Classifier Medium Compression: 25,59 KB

In [3]:
classifier = onnx.load('./onnx_models/medium_fassd__conv341_big__epoch=93.onnx')
onnx.checker.check_model(classifier)

## Detector with Compression and w8a8b8

In [4]:
detector = onnx.load('./onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx')
onnx.checker.check_model(detector)

# Classification Loaders

In [5]:
#classification_loader = classification_data_loader.get_val_loader(shuffle=False)


TEST DFire dataset
DFire Removed wrong images: 0
DFire empty images: 2005
DFire only smoke images: 1186
DFire only fire images: 220
DFire smoke and fire images: 895

Test dataset len: 4306

TEST FASDD UAV dataset
DFire Removed wrong images: 0
DFire empty images: 1997
DFire only smoke images: 846
DFire only fire images: 35
DFire smoke and fire images: 1303

Test FASDD UAV dataset len: 4181

TEST FASDD CV dataset
DFire Removed wrong images: 0
DFire empty images: 6533
DFire only smoke images: 3902
DFire only fire images: 2091
DFire smoke and fire images: 3358

Test FASDD CV dataset len: 15884

Concatenate Test DFire and FASDD UAV datasets
Test dataset len: 8487
Concatenate with FASDD CV dataset
Test dataset len: 24371


# Helper function to convert pytorch tensors to numpy. Useful to handle datatset output.

In [6]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# Evaluate ONNX with F1 Mean, Smoke and Fire

In [7]:
# precision_metric = torchmetrics.classification.MultilabelPrecision(num_labels = config.N_CLASSES, 
#                                                                    threshold = 0.5, 
#                                                                    average = None).to('cpu')
# recall_metric = torchmetrics.classification.MultilabelRecall(num_labels = config.N_CLASSES, 
#                                                              threshold = 0.5, 
#                                                              average = None).to('cpu')
# accuracy_metric = torchmetrics.classification.MultilabelAccuracy(num_labels = config.N_CLASSES, 
#                                                                  threshold = 0.5, 
#                                                                  average = None).to('cpu')
# f1_metric = torchmetrics.classification.MultilabelF1Score(num_labels = config.N_CLASSES, 
#                                                           threshold = 0.5, 
#                                                           average = None).to('cpu')

# f1_metric_mean = torchmetrics.classification.MultilabelF1Score(num_labels = config.N_CLASSES, 
#                                                                threshold = 0.5, 
#                                                                average = 'macro').to('cpu')

### Classifier evaluation funtion

Out of ONNX Model:
- list with 1 np.array
- [ np.array(batch, shape of 1 inference) ]
- to access 1 inference: out[0][0]

In [8]:
# def eval_classifier_onnx(loader, model_name):

#     ort_session = onnxruntime.InferenceSession(model_name, providers=["CPUExecutionProvider"])

#     precision_metric.reset()
#     recall_metric.reset()
#     accuracy_metric.reset()
#     f1_metric.reset()
#     f1_metric_mean.reset()
    
#     loop = tqdm(loader, desc='Validating', leave=True)

#     for batch_idx, (img, label) in enumerate(loop):

#         for idx in range(config.BATCH_SIZE):
            
#             ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img[idx].unsqueeze(dim=0))}
#             yhat = ort_session.run(None, ort_inputs)
#             yhat = np.array(yhat)
#             #yhat = torch.tensor(yhat).squeeze(dim=0)
#             yhat = torch.sigmoid(torch.tensor(yhat).squeeze(dim=0))
#             target = label[idx].unsqueeze(dim=0)

#             precision_metric.update(yhat, target)
#             recall_metric.update(yhat, target)
#             accuracy_metric.update(yhat, target)
#             f1_metric.update(yhat, target)
#             f1_metric_mean.update(yhat, target)
    
#     precision = precision_metric.compute()
#     recall = recall_metric.compute()
#     accuracy = accuracy_metric.compute()
#     f1 = f1_metric.compute()
#     f1_mean = f1_metric_mean.compute()

#     precision_metric.reset()
#     recall_metric.reset()
#     accuracy_metric.reset()
#     f1_metric.reset()
#     f1_metric_mean.reset()

#     print(f'SMOKE -> Precision: {precision[0]:.4f} - Recall: {recall[0]:.4f} - Accuracy: {accuracy[0]:.4f} - F1: {f1[0]:.4f}')
#     print(f'FIRE -> Precision: {precision[1]:.4f} - Recall: {recall[1]:.4f} - Accuracy: {accuracy[1]:.4f} - F1: {f1[1]:.4f}')
#     print(f'Mean F1 Score: {f1_mean.item():.4f}')
    
#     return (
#         {
#         'Accuracy': [accuracy[0].item(), accuracy[1].item()],
#         'Precision': [precision[0].item(), precision[1].item()],
#         'Recall': [recall[0].item(), recall[1].item()],
#         'F1': [f1[0].item(), f1[1].item()],
#         'F1 mean': f1_mean.item(),
#         }
#     )

# Evaluate Classifier Model

In [9]:
# print("\n________________________________ Classifier MEDIUM COMPRESSION _______________________________")
# _ = eval_classifier_onnx(classification_loader, './onnx_models/medium_fassd__conv341_big__epoch=93.onnx')


________________________________ Classifier MEDIUM COMPRESSION _______________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [01:12<00:00,  5.21it/s]

SMOKE -> Precision: 0.9099 - Recall: 0.8943 - Accuracy: 0.9084 - F1: 0.9020
FIRE -> Precision: 0.9032 - Recall: 0.9697 - Accuracy: 0.9565 - F1: 0.9353
Mean F1 Score: 0.9187


### Results with Full Dataset

- SMOKE -> Precision: 0.9099 - Recall: 0.8943 - Accuracy: 0.9084 - F1: 0.9020
- FIRE -> Precision: 0.9032 - Recall: 0.9697 - Accuracy: 0.9565 - F1: 0.9353
- Mean F1 Score: 0.9187

# Detection Loaders

In [10]:
detection_loader = detection_data_loader.get_val_loader()


TEST DFire dataset
DFire Removed wrong images: 0
DFire Removed due to overlapping: 310
DFire Removed due to more than 10: 13

Test DFire dataset len: 3983

TEST FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 377
FASDD Removed due to more than 10: 156

Test FASDD UAV dataset len: 3648

TEST FASDD CV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 317
FASDD Removed due to more than 10: 44

Test FASDD CV dataset len: 15523

Concatenate Test DFire and FASDD UAV datasets
Test dataset len: 7631
Concatenate with FASDD CV dataset
Test dataset len: 23154


# Evaluate Detector

In [11]:
map_metric = MeanAveragePrecision(
    box_format='xyxy',
    iou_thresholds=[config.IOU_THRESHOLD],
    class_metrics=True, # Enables separated metrics for each class
    #average='micro',
    extended_summary=False).to('cpu')

### Score Threshold

In [12]:
SCORE_THRES = 0.2

### Evaluation Function

In [13]:
def eval_detector_onnx(loader, model_name, score_thres):

    ort_session = onnxruntime.InferenceSession(model_name, providers=["CPUExecutionProvider"])

    map_metric.reset()
    
    loop = tqdm(loader, desc='Validating', leave=True)

    for batch_idx, (img, label) in enumerate(loop):

        for idx in range(config.BATCH_SIZE):
            
            ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img[idx].unsqueeze(dim=0))}
            out = ort_session.run(None, ort_inputs)
            
            # out of onnx session is a list: [out_tensor with batch dim] -> [ (1,12,7,7) ] 
            # -> out[0] = (1,12,7,7)
            # -> out[0][0] = (12,7,7)
            #print(f'Out type: {type(out[0])} - Output shape: {out[0].shape}')
            out = torch.tensor(np.array(out[0]))
            out = out.permute(0, 2, 3, 1)
            #print(f'Out shape after permute: {out.shape}')
            #print(f'Out shape after indexing: {out[0].shape}')
            
            # Label should be [xc, yc, w, h, score=1, smoke, fire] in 7x7 square
            #print(f'Label indexed shape: {label[idx].shape}')
            
        # Mean Average Precision
            target_boxes = metrics.get_true_boxes(label[idx].detach().to('cpu'))
            pred_boxes = metrics.get_pred_boxes(
                model_out = out[0].detach().to('cpu'),
                score_threshold=score_thres)
            map_metric.update(preds = pred_boxes, target = target_boxes)
    
    meanAP = map_metric.compute()
    map_metric.reset()

    print(f'Smoke -> AP: {meanAP["map_per_class"][0].item():.4f} - AR: {meanAP["mar_100_per_class"][0].item():.4f}')
    print(f'Fire -> AP: {meanAP["map_per_class"][1].item():.4f} - AR: {meanAP["mar_100_per_class"][1].item():.4f}')
    print(f'mAP: {meanAP["map_50"].item():.4f}')
    
    return (
        {'mAP': meanAP['map_50'].item(),
         'AP': [meanAP['map_per_class'][0].item(), meanAP['map_per_class'][1].item()],
         'AR': [meanAP['mar_100_per_class'][0].item(), meanAP['mar_100_per_class'][1].item()]
        }
    )

In [14]:
print("\n________________________________ Detector MEDIUM COMPRESSION _______________________________")
_ = eval_detector_onnx(
    detection_loader, 
    './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
    SCORE_THRES
)


________________________________ Detector MEDIUM COMPRESSION _______________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [01:19<00:00,  4.55it/s]


Smoke -> AP: 0.6470 - AR: 0.7070
Fire -> AP: 0.6108 - AR: 0.6680
mAP: 0.6289


# Classify 1st + Detection 2nd

In [15]:
def eval_classifier_plus_detector_onnx(
    loader, 
    classifier_model_name,
    detector_model_name,
    score_thres
):

    classify_session = onnxruntime.InferenceSession(classifier_model_name, providers=["CPUExecutionProvider"])
    detect_session = onnxruntime.InferenceSession(detector_model_name, providers=["CPUExecutionProvider"])

    map_metric.reset()
    
    loop = tqdm(loader, desc='Validating', leave=True)

    for batch_idx, (img, label) in enumerate(loop):

        for idx in range(config.BATCH_SIZE):
            
            classify_inputs = {classify_session.get_inputs()[0].name: to_numpy(img[idx].unsqueeze(dim=0))}
            classification_out = classify_session.run(None, classify_inputs)
            # print(f'Smoke pred: {classification_out[0][0][0]}')
            # print(f'Fire pred: {classification_out[0][0][1]}')
            
            # Use Detector if Classifier predicts fire or smoke
            if classification_out[0][0][0] >= 0 or classification_out[0][0][1] >= 0:
                detect_inputs = {detect_session.get_inputs()[0].name: to_numpy(img[idx].unsqueeze(dim=0))}
                out = detect_session.run(None, detect_inputs)
                out = torch.tensor(np.array(out[0]))
                out = out.permute(0, 2, 3, 1)
            else:
                out = torch.zeros(1,7,7,12)           
            
            
        # Mean Average Precision
            target_boxes = metrics.get_true_boxes(label[idx].detach().to('cpu'))
            pred_boxes = metrics.get_pred_boxes(
                model_out = out[0].detach().to('cpu'),
                score_threshold=score_thres)
            map_metric.update(preds = pred_boxes, target = target_boxes)
    
    meanAP = map_metric.compute()
    map_metric.reset()

    print(f'Smoke -> AP: {meanAP["map_per_class"][0].item():.4f} - AR: {meanAP["mar_100_per_class"][0].item():.4f}')
    print(f'Fire -> AP: {meanAP["map_per_class"][1].item():.4f} - AR: {meanAP["mar_100_per_class"][1].item():.4f}')
    print(f'mAP: {meanAP["map_50"]:.4f}')
    
    return (
        {'mAP': meanAP['map_50'].item(),
         'AP': [meanAP['map_per_class'][0].item(), meanAP['map_per_class'][1].item()],
         'AR': [meanAP['mar_100_per_class'][0].item(), meanAP['mar_100_per_class'][1].item()]
        }
    )

### Score Threshold = 0.2

In [16]:
eval_classifier_plus_detector_onnx(
    loader = detection_loader,
    classifier_model_name = './onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detector_model_name = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
    score_thres = 0.2)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [02:58<00:00,  2.03it/s]


Smoke -> AP: 0.6322 - AR: 0.6851
Fire -> AP: 0.6139 - AR: 0.6639
mAP: 0.6231


{'mAP': 0.6230778694152832,
 'AP': [0.6322451233863831, 0.6139106154441833],
 'AR': [0.6851410865783691, 0.6639198660850525]}

### Score Threshold = 0.1

In [17]:
eval_classifier_plus_detector_onnx(
    loader = detection_loader,
    classifier_model_name = './onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detector_model_name = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
    score_thres = 0.1)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [02:57<00:00,  2.04it/s]


Smoke -> AP: 0.6523 - AR: 0.7153
Fire -> AP: 0.6201 - AR: 0.6758
mAP: 0.6362


{'mAP': 0.6361923813819885,
 'AP': [0.6522578001022339, 0.6201269626617432],
 'AR': [0.7153179049491882, 0.6758123636245728]}

### Score Threshold = 0.01 

In [18]:
eval_classifier_plus_detector_onnx(
    loader = detection_loader,
    classifier_model_name = './onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detector_model_name = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
    score_thres = 0.01)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [03:32<00:00,  1.70it/s]


Smoke -> AP: 0.6740 - AR: 0.7727
Fire -> AP: 0.6251 - AR: 0.6896
mAP: 0.6496


{'mAP': 0.649551510810852,
 'AP': [0.6739742755889893, 0.6251287460327148],
 'AR': [0.7726963758468628, 0.6895776987075806]}

### Score Threshold = 0.001 

In [19]:
eval_classifier_plus_detector_onnx(
    loader = detection_loader,
    classifier_model_name = './onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detector_model_name = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
    score_thres = 0.001)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [10:35<00:00,  1.76s/it]


Smoke -> AP: 0.6746 - AR: 0.7843
Fire -> AP: 0.6269 - AR: 0.6904
mAP: 0.6508


{'mAP': 0.6507548093795776,
 'AP': [0.6746267676353455, 0.626882791519165],
 'AR': [0.7843420505523682, 0.6904204487800598]}

### Score Threshold = 0.0001

In [20]:
eval_classifier_plus_detector_onnx(
    loader = detection_loader,
    classifier_model_name = './onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detector_model_name = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
    score_thres = 0.0001)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [13:52<00:00,  2.31s/it]


Smoke -> AP: 0.6746 - AR: 0.7849
Fire -> AP: 0.6269 - AR: 0.6904
mAP: 0.6508


{'mAP': 0.6507548093795776,
 'AP': [0.6746267676353455, 0.626882791519165],
 'AR': [0.7848520874977112, 0.6904204487800598]}